In [ ]:
## Do normal data ingestion from whnece there're coming from

In [2]:
from typing import *
import boto3
import time
import os

# raw-data-validated
# raw-data-unvalidated

## Data movement to Minio(Unvalidated) from producer
# create a bucket
import boto3
from botocore.client import Config

configs = {
    "MINIO_CONFIG": {
        "MINIO_ENDPOINT": "http://localhost:8000",
        "ACCESS_KEY": "nClicM9JjDdMFNYzhqGk",
        "SECRET_KEY": "0w5OD1J6kBtjdGl7CYMovUxPZozcWpS08Yx0EBm0",
    }
}

s3_client = boto3.client(
    "s3",
    endpoint_url=configs["MINIO_CONFIG"]["MINIO_ENDPOINT"],
    aws_access_key_id=configs["MINIO_CONFIG"]["ACCESS_KEY"],
    aws_secret_access_key=configs["MINIO_CONFIG"]["SECRET_KEY"],
    config=Config(signature_version='s3v4')
)


def batch_upload_to_s3(bucket_name,  s3_client, data_dir, file_dir="raw-data-unvalidated"):

    for file_path in os.listdir(data_dir):
        file_path = os.path.join(data_dir, file_path)
        file_name = os.path.basename(file_path)

        s3_key = f"{file_dir}/{file_name}" 

        # Upload the file
        s3_client.upload_file(file_path, bucket_name, s3_key)
        print(f'Uploaded {file_name} to {bucket_name}/{file_dir}.')

data = "/home/alijoe/Documents/Projects/e2e_ml_project/credit_card_fraud_detection/simulated-data-raw/data"
batch_upload_to_s3("ccf-detection-demo", s3_client, data)

Uploaded 2018-08-31.pkl to ccf-detection-demo/raw-data-unvalidated.
Uploaded 2018-05-31.pkl to ccf-detection-demo/raw-data-unvalidated.
Uploaded 2018-06-21.pkl to ccf-detection-demo/raw-data-unvalidated.
Uploaded 2018-09-20.pkl to ccf-detection-demo/raw-data-unvalidated.
Uploaded 2018-09-23.pkl to ccf-detection-demo/raw-data-unvalidated.
Uploaded 2018-04-14.pkl to ccf-detection-demo/raw-data-unvalidated.
Uploaded 2018-04-27.pkl to ccf-detection-demo/raw-data-unvalidated.
Uploaded 2018-05-08.pkl to ccf-detection-demo/raw-data-unvalidated.
Uploaded 2018-07-31.pkl to ccf-detection-demo/raw-data-unvalidated.
Uploaded 2018-06-04.pkl to ccf-detection-demo/raw-data-unvalidated.
Uploaded 2018-07-12.pkl to ccf-detection-demo/raw-data-unvalidated.
Uploaded 2018-06-18.pkl to ccf-detection-demo/raw-data-unvalidated.
Uploaded 2018-09-01.pkl to ccf-detection-demo/raw-data-unvalidated.
Uploaded 2018-05-17.pkl to ccf-detection-demo/raw-data-unvalidated.
Uploaded 2018-08-01.pkl to ccf-detection-demo/ra

#### Create GX Validations Schema

In [ ]:
## Data Quality Check and Validation using Great Expectation

In [3]:
import great_expectations as gx
import great_expectations as gx
from great_expectations.data_context.types.base import (
    DataContextConfig,
    DatasourceConfig,
    FilesystemStoreBackendDefaults,
    S3StoreBackendDefaults
)

from great_expectations.core.batch import RuntimeBatchRequest


# Define your store backends amd their location (preferred in S3)

# N.B: If you get an error, create your gx bucket before you run this

expectations_s3_store_backend = {
    "class_name": "TupleS3StoreBackend",
    "bucket": "great-expectation-stores-new",
    "prefix": "expectations",
    "boto3_options": {
        "endpoint_url": "http://localhost:8000",
        "aws_access_key_id": "nClicM9JjDdMFNYzhqGk",
        "aws_secret_access_key": "0w5OD1J6kBtjdGl7CYMovUxPZozcWpS08Yx0EBm0",
    }
}

validations_s3_store_backend = {
    "class_name": "TupleS3StoreBackend",
    "bucket": "great-expectation-stores-new",
    "prefix": "uncommitted/validations/",
    "boto3_options": {
        "endpoint_url": "http://localhost:8000",
        "aws_access_key_id": "nClicM9JjDdMFNYzhqGk",
        "aws_secret_access_key": "0w5OD1J6kBtjdGl7CYMovUxPZozcWpS08Yx0EBm0",
    }
}

evaluation_parameter_s3_store_backend = {
    "class_name": "TupleS3StoreBackend",
    "bucket": "great-expectation-stores-new",
    "prefix": "evaluations/",
    "boto3_options": {
        "endpoint_url": "http://localhost:8000",
        "aws_access_key_id": "nClicM9JjDdMFNYzhqGk",
        "aws_secret_access_key": "0w5OD1J6kBtjdGl7CYMovUxPZozcWpS08Yx0EBm0",
    }
}

profiler_s3_store_backend = {
  "class_name": "TupleS3StoreBackend",
  "bucket": "great-expectation-stores-new",
  "suppress_store_backend_id": True,
  "prefix": "profilers/",
  "boto3_options": {
        "endpoint_url": "http://localhost:8000",
        "aws_access_key_id": "nClicM9JjDdMFNYzhqGk",
        "aws_secret_access_key": "0w5OD1J6kBtjdGl7CYMovUxPZozcWpS08Yx0EBm0",
    }
}

checkpoint_s3_store_backend = {
  "class_name": "TupleS3StoreBackend",
  "bucket": "great-expectation-stores-new",
  "suppress_store_backend_id": True,
  "prefix": "checkpoints/",
  "boto3_options": {
        "endpoint_url": "http://localhost:8000",
        "aws_access_key_id": "nClicM9JjDdMFNYzhqGk",
        "aws_secret_access_key": "0w5OD1J6kBtjdGl7CYMovUxPZozcWpS08Yx0EBm0",
    }
}

data_docs_sites_s3_store_backend = {
    "class_name": "TupleS3StoreBackend",
    "bucket": "great-expectation-stores-new",
    "prefix": "data-docs/",
    "boto3_options": {
        "endpoint_url": "http://localhost:8000",
        "aws_access_key_id": "nClicM9JjDdMFNYzhqGk",
        "aws_secret_access_key": "0w5OD1J6kBtjdGl7CYMovUxPZozcWpS08Yx0EBm0",
    }
}


In [10]:
# Step 2: Configure the Data Context

## DataContextConfig is just same way you create `great_expectation.yaml`, but now everything is in code
## so rather than doing `great_expectation init`, just use the DataContextConfig class
## to define everything that you'd see in a `great_expectation.yaml` file.
## down in the notebook, there is a `great_expectation.yaml` file example

data_context_config = DataContextConfig(
    datasources={
        "minio_datasource": DatasourceConfig(
            class_name="Datasource",
            execution_engine={
                "class_name": "PandasExecutionEngine"
            },
            data_connectors={
                "default_runtime_data_connector_name": {
                    "class_name": "RuntimeDataConnector",
                    "module_name": "great_expectations.datasource.data_connector",
                    "batch_identifiers": ["default_identifier_name"],
                },
                "default_inferred_data_connector_name": {
                    "class_name": "InferredAssetS3DataConnector",
                    "name": "minio_asset",
                    "bucket": "ccf-detection-demo/",
                    "prefix": "raw-data-unvalidated",
                    "default_regex": {
                        "group_names": ["data_asset_name"],
                        "pattern": r"(.*)\.pkl",
                    },
                    "boto3_options": {
                        "aws_access_key_id": "nClicM9JjDdMFNYzhqGk",
                        "aws_secret_access_key": "0w5OD1J6kBtjdGl7CYMovUxPZozcWpS08Yx0EBm0",
                        "endpoint_url": "http://localhost:8000",
                    },
                },
            },
        )
    },
    expectations_store_name = "expectations_S3_store",
    validations_store_name = "validations_store_name",
    evaluation_parameter_store_name = "evaluation_parameter_store_name",
    checkpoint_store_name = "checkpoint_store_name",
    profiler_store_name = "profiler_store_name",
    plugins_directory = "plugins/", # remove this line should you get an error

    ## Define the expectatinos stores to use ....
    stores={
        "expectations_S3_store": {
            "class_name": "ExpectationsStore",
            "store_backend": expectations_s3_store_backend
        },
        "validations_store_name": {
            "class_name": "ValidationsStore",
            "store_backend": validations_s3_store_backend
        },
        "evaluation_parameter_store_name": {
            "class_name": "EvaluationParameterStore",
            "store_backend": evaluation_parameter_s3_store_backend
        },
        "checkpoint_store_name": {
            "class_name": "CheckpointStore",
            "store_backend": checkpoint_s3_store_backend
        },
        "profiler_store_name": {
            "class_name": "ProfilerStore",
            "store_backend": profiler_s3_store_backend
        }
    },
    data_docs_sites={
        "s3_site": {
            "class_name": "SiteBuilder",
            "show_how_to_buttons": True,
            "site_index_builder": {
                "class_name": "DefaultSiteIndexBuilder"
            },
            "store_backend": data_docs_sites_s3_store_backend
        }
    }
)

# Initialize the context
# context = BaseDataContext(project_config=data_context_config)
context = gx.get_context(project_config=data_context_config)


In [ ]:
# datasource = context.get_datasource("minio_datasource") #.get_ba get_asset("minio_asset")
boto3_option = {
        "aws_access_key_id": "nClicM9JjDdMFNYzhqGk",
        "aws_secret_access_key": "0w5OD1J6kBtjdGl7CYMovUxPZozcWpS08Yx0EBm0",
        "endpoint_url": "http://localhost:8000",
}

datasource = context.sources.add_pandas_s3(
        name="new_datasource_", bucket="ccf-detection-demo", boto3_options=boto3_option
)

In [31]:
data_asset = datasource.add_pickle_asset(
    name = "ccf-asset-a",
    s3_prefix=  "raw-data-unvalidated/",
    batching_regex = r"\d{4}-(\d{2})-\d{2}\.pkl"
)


In [32]:
batch_request = data_asset.build_batch_request()
batch_request

BatchRequest(datasource_name='new_datasource_', data_asset_name='ccf-asset-a', options={})

In [ ]:


context.add_or_update_expectation_suite(expectation_suite_name="test_suite")
validator = context.get_validator(
    batch_request=batch_request, expectation_suite_name="test_suite"
)

In [11]:
datassource = context.get_datasource("minio_datasource")
datassource._add_asset

In [7]:
context = gx.get_context(mode="file")

data_source = context.data_sources.add_pandas_s3(
    name="minio_datasource_2", bucket="ccf-detection-demo", boto3_options={
                        "aws_access_key_id": "nClicM9JjDdMFNYzhqGk",
                        "aws_secret_access_key": "0w5OD1J6kBtjdGl7CYMovUxPZozcWpS08Yx0EBm0",
                        "endpoint_url": "http://localhost:8000",
                    }
)

AttributeError: 'FileDataContext' object has no attribute 'data_sources'

In [33]:
# Create a batch of data
batch_request = RuntimeBatchRequest(
    datasource_name="minio_datasource",
    data_connector_name="default_runtime_data_connector_name",
    data_asset_name="ccf_detection",  # This can be anything that identifies this data_asset for you
    runtime_parameters={"path": "s3a://ccf-detection-demo/raw-data-unvalidated/2018-04-01.pkl"}, ## Look how to make this more better, red teh docs
    batch_identifiers={"default_identifier_name": "default_identifier"},
)


TypeError: RuntimeBatchRequest.__init__() missing 1 required positional argument: 'runtime_parameters'

In [ ]:
context.sources.

In [34]:
datasource = context.sources.add_or_update_spark_s3(
    name="minio_ccf_datasource",
    bucket="ccf-detection-demo",
    boto3_options={
        "aws_access_key_id": "nClicM9JjDdMFNYzhqGk",
        "aws_secret_access_key": "0w5OD1J6kBtjdGl7CYMovUxPZozcWpS08Yx0EBm0",
        "endpoint_url": "http://localhost:8000",
    })

Picked up _JAVA_OPTIONS: -Dawt.useSystemAAFontSettings=on -Dswing.aatext=true
Picked up _JAVA_OPTIONS: -Dawt.useSystemAAFontSettings=on -Dswing.aatext=true
25/01/20 06:57:34 WARN Utils: Your hostname, katana resolves to a loopback address: 127.0.1.1; using 192.168.0.116 instead (on interface wlan0)
25/01/20 06:57:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/20 06:57:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
asset = datasource.add

In [ ]:
asset = datasource.add_csv_asset(
    name="csv_taxi_s3_asset",
    batching_regex=r".*_(?P<year>\d{4})\.csv",
    header=True,
    infer_schema=True,
)

In [19]:
context.add_or_update_expectation_suite(
    expectation_suite_name="ccf_detection_suite") #, overwrite_existing=True
#)

{
  "expectation_suite_name": "ccf_detection_suite",
  "ge_cloud_id": null,
  "expectations": [],
  "data_asset_type": null,
  "meta": {
    "great_expectations_version": "0.18.19"
  }
}

In [29]:
validator = context.get_validator(
    batch_request=batch_request, expectation_suite_name="ccf_detection_suite"
)

print (validator)

In [21]:
validator.head()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO
0,0,2018-04-01 00:00:31,596,3156,57.16,31,0,0,0
1,1,2018-04-01 00:02:10,4961,3412,81.51,130,0,0,0
2,2,2018-04-01 00:07:56,2,1365,146.00,476,0,0,0
3,3,2018-04-01 00:09:29,4128,8737,64.49,569,0,0,0
4,4,2018-04-01 00:10:34,927,9906,50.99,634,0,0,0


In [22]:
## Using automated process to generate expectation suite
### This is a good starting point, then you continue from here to create addition ones
from great_expectations.profile.user_configurable_profiler import UserConfigurableProfiler

profiler = UserConfigurableProfiler(profile_dataset=validator)
suite = profiler.build_suite()


Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/7 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

...

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/7 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/7 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Creating an expectation suite with the following expectations:

Table-Level Expectations
expect_table_columns_to_match_ordered_list
expect_table_row_count_to_be_between

Expectations by Column
Column Name: CUSTOMER_ID | Column Data Type: INT | Cardinality: VERY_MANY
expect_column_max_to_be_between
expect_column_mean_to_be_between
expect_column_median_to_be_between
expect_column_min_to_be_between
expect_column_proportion_of_unique_values_to_be_between
expect_column_quantile_values_to_be_between
expect_column_values_to_be_in_type_list
expect_column_values_to_not_be_null


Column Name: TERMINAL_ID | Column Data Type: INT | Cardinality: VERY_MANY
expect_column_max_to_be_between
expect_column_mean_to_be_between
expect_column_median_to_be_between
expect_column_min_to_be_between
expect_column_proportion_of_unique_values_to_be_between
expect_column_quantile_values_to_be_between
expect_column_values_to_be_in_type_list
expect_column_values_to_not_be_null


Column Name: TRANSACTION_ID | Column Da

In [23]:
## View the suite before saving
import pprint
suite = validator.get_expectation_suite()
pprint.pprint(suite.to_json_dict())

{'data_asset_type': None,
 'expectation_suite_name': 'ccf_detection_suite',
 'expectations': [{'expectation_type': 'expect_table_columns_to_match_ordered_list',
                   'kwargs': {'column_list': ['TRANSACTION_ID',
                                              'TX_DATETIME',
                                              'CUSTOMER_ID',
                                              'TERMINAL_ID',
                                              'TX_AMOUNT',
                                              'TX_TIME_SECONDS',
                                              'TX_TIME_DAYS',
                                              'TX_FRAUD',
                                              'TX_FRAUD_SCENARIO']},
                   'meta': {}},
                  {'expectation_type': 'expect_table_row_count_to_be_between',
                   'kwargs': {'max_value': 9488, 'min_value': 9488},
                   'meta': {}},
                  {'expectation_type': 'expect_column_min_to_be_bet

In [25]:
# Save the generated Expectation Suite
context.save_expectation_suite(expectation_suite=suite, expectation_suite_name="ccf_detection_suite")


'expectations/ccf_detection_suite.json'

In [26]:
context.build_data_docs()


{'s3_site': 'http://localhost:8000/great-expectation-stores/data-docs/index.html'}

#### Validate unvcalidated data with GX

In [31]:
validation_results = validator.validate()

print(validation_results)

TypeError: key must be an instance of ValidationMetricIdentifier, not <class 'str'>